In [1]:
import pandas as pd
import numpy as np

In [47]:
import requests
import pandas as pd
from datetime import date

today = date.today().strftime('%Y-%m-%d')
url = "https://api.finmindtrade.com/api/v3/data"
parameter = {
    "dataset": "TaiwanStockPrice",
    "stock_id": "2368",
    "date": "2019-01-01",
    "end_date": today,
}
resp = requests.get(url, params=parameter)
data = resp.json()
data = pd.DataFrame(data["data"])

df = data[['date','open','max','min','close','Trading_Volume']]
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
df['Volume'] = df['Volume']

In [48]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [55]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 20
    n3 = 60

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.sma3 = self.I(SMA, self.data.Close, self.n3)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if (crossover(self.sma1, self.sma2)) and (self.sma2[-1] > self.sam3[-1]):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1) and  (self.sma2[-1] < self.sam3[-1]):
            self.position.close()
            self.sell()

In [56]:
from backtesting import Backtest

bt = Backtest(df, SmaCross, cash=10_000, commission=.001425)
stats = bt.run()
stats

Start                     2019-01-02 00:00:00
End                       2020-12-31 00:00:00
Duration                    729 days 00:00:00
Exposure Time [%]                     94.8665
Equity Final [$]                       6600.1
Equity Peak [$]                         10000
Return [%]                            -33.999
Buy & Hold Return [%]                  284.03
Return (Ann.) [%]                     -19.346
Volatility (Ann.) [%]                  45.248
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                    -67.3886
Avg. Drawdown [%]                    -67.3886
Max. Drawdown Duration      686 days 00:00:00
Avg. Drawdown Duration      686 days 00:00:00
# Trades                                   26
Win Rate [%]                          26.9231
Best Trade [%]                        55.9989
Worst Trade [%]                      -21.8466
Avg. Trade [%]                    

In [57]:
bt.plot()